In [1]:
import numpy as np
import pandas as pd
from Bio import SeqIO
from Bio.Seq import MutableSeq

In [2]:
import gzip
from collections import namedtuple

Feature = namedtuple('Feature', ['id',
                                 'ftype',
                                 'chromosome',
                                 'start',
                                 'end',
                                 'strand',
                                 'gene',
                                 'product'])

def parse_gff(file_name):
    # output dict
    # key: feature ID
    # value: Feature NamedTuple
    features = {}

    with gzip.open(file_name, 'rt') as gff:
        for line in gff:
            if line.lstrip().startswith('##FASTA'):
                # start of FASTA entries, end of file
                break

            elif line.lstrip().startswith('#'):
                # comment, ignore
                continue

            # should be a valid GFF3 line
            entries = line.split('\t')

            try:
                ftype = entries[2]

                chrom = entries[0]
                start = int(entries[3])
                end = int(entries[4])
                strand = entries[6]

                # integer takes up less space
                if strand == '+':
                    strand = 1
                else:
                    strand = -1

                # fetch the feature ID from the last field
                ID = None
                for entry in entries[8].split(';'):
                    if entry.startswith('ID=') and '=' in entry:
                        ID = entry.split('=')[1]

                # could not find it, skip this entry
                if ID is None:
                    continue

                # fetch the gene name
                gene = np.nan
                for entry in entries[8].split(';'):
                    if entry.startswith('gene=') and '=' in entry:
                        gene = entry.split('=')[1]

                product = np.nan
                for entry in entries[8].split(';'):
                    if entry.startswith('product=') and '=' in entry:
                        product = entry.split('=')[1]

                # save the relevant details
                features[ID] = Feature(ID, ftype, chrom, start, end, strand, gene, product)

            except Exception as e:
                # not distinguishing between exceptions
                # not great behaviour
                logger.warning(f'{e}, skipping line "{line.rstrip()}" from {file_name}')
                continue

    res = []
    for feat in features.values():
        for i, position in enumerate(range(feat.start, feat.end+1)):
            res.append( (feat.chromosome, feat.ftype, position, i+1, feat.strand, feat.id, feat.gene, feat.product) )
    r = pd.DataFrame(res,
                     columns=['chromosome', 'ftype', 'position', 'feature_position', 'strand', 'id', 'gene', 'product'])

    r['codon'] = np.nan
    r['codon'] = (r['feature_position'] % 3).astype(int)
    r.loc[r[r['codon'] == 0].index, 'codon'] = 3
    r.loc[r[r['ftype'] != 'CDS'].index, 'codon'] = np.nan

    r['feature_codon'] = (r['feature_position'] / 3).astype(int)+1
    r.loc[r[r['ftype'] != 'CDS'].index, 'feature_codon'] = np.nan
    r.loc[r[r['codon'] == 3.0].index, 'feature_codon'] -= 1

    r = r[r['ftype'].isin(['five_prime_UTR', 'three_prime_UTR', 'CDS'])]

    return r

r = parse_gff('../data/GCF_009858895.2_ASM985889v3_genomic.gff.gz')

In [3]:
r = r[r['gene'] == 'S'].copy()

In [4]:
r

,chromosome,ftype,position,feature_position,strand,id,gene,product,codon,feature_codon
111145,NC_045512.2,CDS,21563,1,1,cds-YP_009724390.1,S,surface glycoprotein,1.0,1.0
111146,NC_045512.2,CDS,21564,2,1,cds-YP_009724390.1,S,surface glycoprotein,2.0,1.0
111147,NC_045512.2,CDS,21565,3,1,cds-YP_009724390.1,S,surface glycoprotein,3.0,1.0
111148,NC_045512.2,CDS,21566,4,1,cds-YP_009724390.1,S,surface glycoprotein,1.0,2.0
111149,NC_045512.2,CDS,21567,5,1,cds-YP_009724390.1,S,surface glycoprotein,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...
114962,NC_045512.2,CDS,25380,3818,1,cds-YP_009724390.1,S,surface glycoprotein,2.0,1273.0
114963,NC_045512.2,CDS,25381,3819,1,cds-YP_009724390.1,S,surface glycoprotein,3.0,1273.0
114964,NC_045512.2,CDS,25382,3820,1,cds-YP_009724390.1,S,surface glycoprotein,1.0,1274.0
114965,NC_045512.2,CDS,25383,3821,1,cds-YP_009724390.1,S,surface glycoprotein,2.0,1274.0


In [5]:
r[(r['feature_codon'] >= 319) & (r['feature_codon'] <= 540)]

,chromosome,ftype,position,feature_position,strand,id,gene,product,codon,feature_codon
112099,NC_045512.2,CDS,22517,955,1,cds-YP_009724390.1,S,surface glycoprotein,1.0,319.0
112100,NC_045512.2,CDS,22518,956,1,cds-YP_009724390.1,S,surface glycoprotein,2.0,319.0
112101,NC_045512.2,CDS,22519,957,1,cds-YP_009724390.1,S,surface glycoprotein,3.0,319.0
112102,NC_045512.2,CDS,22520,958,1,cds-YP_009724390.1,S,surface glycoprotein,1.0,320.0
112103,NC_045512.2,CDS,22521,959,1,cds-YP_009724390.1,S,surface glycoprotein,2.0,320.0
...,...,...,...,...,...,...,...,...,...,...
112760,NC_045512.2,CDS,23178,1616,1,cds-YP_009724390.1,S,surface glycoprotein,2.0,539.0
112761,NC_045512.2,CDS,23179,1617,1,cds-YP_009724390.1,S,surface glycoprotein,3.0,539.0
112762,NC_045512.2,CDS,23180,1618,1,cds-YP_009724390.1,S,surface glycoprotein,1.0,540.0
112763,NC_045512.2,CDS,23181,1619,1,cds-YP_009724390.1,S,surface glycoprotein,2.0,540.0


In [6]:
c = SeqIO.read('../data/NC_045512.2.fasta', 'fasta')

In [7]:
aa = set([446, 452, 460, 486, 490, 498, 501])

In [8]:
nt2aa = r[r['feature_codon'].isin(aa)].set_index('position')['feature_codon'].to_dict()

In [9]:
pos = set(r[r['feature_codon'].isin(aa)]['position'].values)

In [10]:
names = [f'2023-{x:02d}' for x in range(5, 9)]

In [11]:
res = []
res1 = []
for j, name in enumerate(names):
    l = pd.read_csv(f'../out/time-lineages/{name}.tsv', sep='\t')
    for i, data in l[['aaSubstitutions', 'substitutions']].iterrows():
        aas = {x for x in data.loc['aaSubstitutions'].split(',') if x.split(':')[0] == 'S' and int(x.split(':')[1][1:-1]) in aa}
        d = {'name': name, 'seq': f'{j}-{i}'}
        d1 = {'name': name, 'seq': f'{j}-{i}'}
        for v in aas:
            p = int(v.split(':')[1][1:-1])
            nts = {x for x in data.loc['substitutions'].split(',')
                   if int(x[1:-1]) in nt2aa and nt2aa[int(x[1:-1])] == int(v.split(':')[1][1:-1])}
            muts = ','.join(sorted(nts))
            d[p] = muts
            d1[p] = v
        res.append(d)
        res1.append(d1)
d1 = pd.DataFrame(res)
d2 = pd.DataFrame(res1)

In [12]:
d1 = d1.fillna('')

In [13]:
d2 = d2.fillna('')

In [14]:
d1.groupby(sorted(aa))['seq'].count().reset_index()

,446,452,460,486,490,498,501,seq
0,,,,,,,,42
1,,,,"T23018C,T23019C",T23031C,A23055G,A23063T,19
2,,,,T23018G,,A23055G,A23063T,1
3,,,T22942G,,T23031C,A23055G,A23063T,3
4,,,T22942G,"T23018C,T23019C",T23031C,A23055G,A23063T,4
5,,T22917G,,"T23018C,T23019C",T23031C,A23055G,A23063T,1
6,,T22917G,,T23018G,,A23055G,A23063T,27
7,,T22917G,T22942A,T23018G,,A23055G,A23063T,8
8,G22898A,,,"T23018C,T23019C",T23031C,A23055G,A23063T,5
9,G22898A,,T22942G,,,,,1


In [15]:
d2.groupby(sorted(aa))['seq'].count().reset_index()

,446,452,460,486,490,498,501,seq
0,,,,,,,,42
1,,,,S:F486P,S:F490S,S:Q498R,S:N501Y,19
2,,,,S:F486V,,S:Q498R,S:N501Y,1
3,,,S:N460K,,S:F490S,S:Q498R,S:N501Y,3
4,,,S:N460K,S:F486P,S:F490S,S:Q498R,S:N501Y,4
5,,S:L452R,,S:F486P,S:F490S,S:Q498R,S:N501Y,1
6,,S:L452R,,S:F486V,,S:Q498R,S:N501Y,27
7,,S:L452R,S:N460K,S:F486V,,S:Q498R,S:N501Y,8
8,S:G446S,,,S:F486P,S:F490S,S:Q498R,S:N501Y,5
9,S:G446S,,S:N460K,,,,,1


In [16]:
d1.groupby([498, 501])['seq'].count().reset_index()

,498,501,seq
0,,,45
1,,A23063T,1
2,A23055G,A23063T,3228
3,"A23055G,C23054A",A23063T,2


In [17]:
d2.groupby([498, 501])['seq'].count().reset_index()

,498,501,seq
0,,,45
1,,S:N501Y,1
2,S:Q498R,S:N501Y,3230


In [18]:
d1.groupby([446, 460])['seq'].count().reset_index()

,446,460,seq
0,,,90
1,,T22942A,8
2,,T22942G,7
3,G22898A,,15
4,G22898A,A22940C,5
5,G22898A,T22942G,3151


In [19]:
d2.groupby([446, 460])['seq'].count().reset_index()

,446,460,seq
0,,,90
1,,S:N460K,15
2,S:G446S,,15
3,S:G446S,S:N460H,5
4,S:G446S,S:N460K,3151


In [20]:
d1.groupby([446, 486])['seq'].count().reset_index()

,446,486,seq
0,,,45
1,,"T23018C,T23019C",24
2,,T23018G,36
3,G22898A,,43
4,G22898A,T23018A,2
5,G22898A,"T23018C,T23019C",3039
6,G22898A,T23019C,87


In [21]:
d2.groupby([446, 486])['seq'].count().reset_index()

,446,486,seq
0,,,45
1,,S:F486P,24
2,,S:F486V,36
3,S:G446S,,43
4,S:G446S,S:F486I,2
5,S:G446S,S:F486P,3039
6,S:G446S,S:F486S,87


In [22]:
d1.groupby([452, 490])['seq'].count().reset_index()

,452,490,seq
0,,,47
1,,"T23030C,T23031C",12
2,,"T23030G,T23031C",3
3,,T23031C,3085
4,C22916A,,4
5,"C22916A,T22917G",,15
6,T22917A,T23031C,2
7,T22917G,,105
8,T22917G,T23031C,3


In [23]:
d2.groupby([452, 490])['seq'].count().reset_index()

,452,490,seq
0,,,47
1,,S:F490A,3
2,,S:F490P,12
3,,S:F490S,3085
4,S:L452M,,4
5,S:L452Q,S:F490S,2
6,S:L452R,,120
7,S:L452R,S:F490S,3


In [24]:
d2.groupby([446, 486, 498, 501])['seq'].count().reset_index()

,446,486,498,501,seq
0,,,,,42
1,,,S:Q498R,S:N501Y,3
2,,S:F486P,S:Q498R,S:N501Y,24
3,,S:F486V,S:Q498R,S:N501Y,36
4,S:G446S,,,,1
5,S:G446S,,S:Q498R,S:N501Y,42
6,S:G446S,S:F486I,S:Q498R,S:N501Y,2
7,S:G446S,S:F486P,,,2
8,S:G446S,S:F486P,,S:N501Y,1
9,S:G446S,S:F486P,S:Q498R,S:N501Y,3036


In [25]:
d2.groupby([452, 490, 498, 501])['seq'].count().reset_index()

,452,490,498,501,seq
0,,,,,43
1,,,S:Q498R,S:N501Y,4
2,,S:F490A,S:Q498R,S:N501Y,3
3,,S:F490P,S:Q498R,S:N501Y,12
4,,S:F490S,,,2
5,,S:F490S,,S:N501Y,1
6,,S:F490S,S:Q498R,S:N501Y,3082
7,S:L452M,,S:Q498R,S:N501Y,4
8,S:L452Q,S:F490S,S:Q498R,S:N501Y,2
9,S:L452R,,S:Q498R,S:N501Y,120


In [26]:
d2.groupby([446, 452, 460, 486, 490, 498, 501])['seq'].count().reset_index()

,446,452,460,486,490,498,501,seq
0,,,,,,,,42
1,,,,S:F486P,S:F490S,S:Q498R,S:N501Y,19
2,,,,S:F486V,,S:Q498R,S:N501Y,1
3,,,S:N460K,,S:F490S,S:Q498R,S:N501Y,3
4,,,S:N460K,S:F486P,S:F490S,S:Q498R,S:N501Y,4
5,,S:L452R,,S:F486P,S:F490S,S:Q498R,S:N501Y,1
6,,S:L452R,,S:F486V,,S:Q498R,S:N501Y,27
7,,S:L452R,S:N460K,S:F486V,,S:Q498R,S:N501Y,8
8,S:G446S,,,S:F486P,S:F490S,S:Q498R,S:N501Y,5
9,S:G446S,,S:N460K,,,,,1


In [27]:
s = MutableSeq(str(c.seq))

In [28]:
muts = ['G22898A',
'T22917A',
'T22917G',
'A22940C',
'T23018A',
'T23019C',
'T23018G',
'T23031C',
'T23030C T23031C',
'A23055G',
'A23063T',
'A23055G A23063T',
'G22898A A22940C',
'G22898A T23018A',
'G22898A T23019C',
'G22898A T23018G',
'T22917A T23031C',
'T22917G T23031C',
'T22917G T23030C T23031C',
'G22898A A22940C A23055G A23063T',
'G22898A T23018A A23055G A23063T',
'G22898A T23018G A23055G A23063T',
'T23018G A23055G A23063T',
'G22898A T23019C A23055G A23063T',
'T22917A T23031C A23055G A23063T',
'T22917G T23031C A23055G A23063T',
'T22917G A23055G A23063T',
'T22917G T23030C T23031C A23055G A23063T',
'T23030C T23031C A23055G A23063T']

In [29]:
full = []
spike = []
rbd = []
for i, mut in enumerate(muts):
    s = MutableSeq(str(c.seq))
    for m in mut.split():
        pos = int(m[1:-1])
        old = m[0]
        new = m[-1]
        assert s[pos-1] == old
        s[pos-1] = new
    full.append(s)
    spike.append(s[21563-1:25384])
    rbd.append(s[22517-1:23182])

In [30]:
f = open('full.fasta', 'w')
for i, s in enumerate(full):
    j = i + 1
    f.write(f'>epi-{j:02d}\n{str(s)}\n')
f.close()

In [31]:
f = open('spike.fasta', 'w')
for i, s in enumerate(spike):
    j = i + 1
    f.write(f'>epi-{j:02d}\n{str(s)}\n')
f.close()

In [32]:
f = open('rbd.fasta', 'w')
for i, s in enumerate(rbd):
    j = i + 1
    f.write(f'>epi-{j:02d}\n{str(s)}\n')
f.close()